In [1]:
# (ignore this) imports and utility functions

import re
import json
import toolkit

from textwrap import dedent
from IPython.display import clear_output, display, Markdown


def render_markdown(text):
    if text is None:
        text = ""

    pattern = r"```thinking\n(.*?)\n```"

    def replacement(match):
        content = match.group(1)
        return f"<blockquote>\n{content}\n</blockquote>\n\n"

    text = re.sub(pattern, replacement, text, flags=re.DOTALL)

    text = re.sub(r"```function_spec\b", "```json", text)
    text = re.sub(r"```function_call\b", "```json", text)
    text = re.sub(r"```function_output\b", "```json", text)

    display(Markdown(text))


def render_json(data):
    return render_markdown(f"```json\n{json.dumps(data, indent=2)}\n```\n")


def render_live(stream):
    response = ""
    render_markdown("> Loading...")

    for chunk in stream:
        response += chunk
        clear_output(wait=True)
        render_markdown(response)

    return response

In [2]:
model = "gemma3:27b"

In [3]:
task = dedent("""
    Generate a simple python function that fetches the weather for any place using the wttr.in API. The API is
    simple, just pass the name of the place like so:
    
        "http://wttr.in/<place>?format=(%l) %C and %t (feels %f) with winds at %w"

    The place can be a place name (Mumbai), an airport code (BOM), or an iconic location (Gateway of India).

    Do not generate examples that use the function. Do not print the result, return it from the function. Do not
    handle errors by printing or ignoring them, raise a custom exception with a user-friendly message instead.
""")

In [4]:
messages = [{"role": "user", "content": task}]

In [5]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

```python
import requests

class WeatherFetchError(Exception):
    """Custom exception for weather fetching errors."""
    pass

def get_weather(place):
    """
    Fetches the weather for a given place using the wttr.in API.

    Args:
        place (str): The name of the place, airport code, or iconic location.

    Returns:
        str: The weather information from the API.

    Raises:
        WeatherFetchError: If the request to the API fails or returns an error.
    """
    url = f"http://wttr.in/{place}?format=(%l) %C and %t (feels %f) with winds at %w"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        return response.text
    except requests.exceptions.RequestException as e:
        raise WeatherFetchError(f"Failed to fetch weather for {place}: {e}")
```

In [6]:
regex = re.compile(toolkit.functions.code_block_regex, re.VERBOSE | re.DOTALL)

definitions = []
for match in regex.finditer(complete_response):
    if match.group("code_class") == "python":
        definitions.append(match.group("code_content"))

weather_function = definitions[0]

In [7]:
# (ignore this) it has functions for testing

# weather_function = 'import requests\n\nclass WeatherFetchError(Exception):\n    """Custom exception for weather fetching errors."""\n    pass\n\ndef get_weather(place):\n    """\n    Fetches the weather for a given place using the wttr.in API.\n\n    Args:\n        place (str): The name of the place, airport code, or iconic location.\n\n    Returns:\n        str: The weather information as a string.\n\n    Raises:\n        WeatherFetchError: If the request to wttr.in fails.\n    """\n    url = f"http://wttr.in/{place}?TF"\n    try:\n        response = requests.get(url)\n        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)\n        return response.text\n    except requests.exceptions.RequestException as e:\n        raise WeatherFetchError(f"Failed to fetch weather for {place}. Error: {e}")'
# weather_function = 'import requests\n\nclass WeatherFetchError(Exception):\n    """Custom exception for weather fetching errors."""\n    pass\n\ndef get_weather(place):\n    """\n    Fetches the weather for a given place using the wttr.in API.\n\n    Args:\n        place (str): The name of the place, airport code, or iconic location.\n\n    Returns:\n        str: The weather information as a string.\n\n    Raises:\n        WeatherFetchError: If the request to wttr.in fails.\n    """\n    raise WeatherFetchError(f"Failed to fetch weather for {place}. Error: none :)")'

In [8]:
await toolkit.code.create_sandbox()

In [9]:
await toolkit.code.define_function(weather_function)

In [22]:
output = await toolkit.code.execute_function("get_weather", {"place": "Pune"})
render_markdown(output["result"])

(Pune) Patchy rain nearby and +24°C (feels +26°C) with winds at →22km/h

In [11]:
with open("schemas/function.schema.json") as file:
    schema = json.load(file)

In [12]:
task = dedent(f"""
    Generate a function specification for the function you just generated. Make sure it is produced in a code block
    with the language set to `function_spec` instead of `json`. Make sure to follow the below schema:

    {json.dumps(schema, indent=2)}
""")

In [13]:
messages.append({"role": "user", "content": task})

In [14]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

```json
{
  "name": "get_weather",
  "description": "Fetches the current weather information for a given location using the wttr.in API. The location can be a city name, airport code, or iconic landmark.",
  "parameters": {
    "type": "object",
    "properties": {
      "place": {
        "type": "string",
        "description": "The name of the place (city, airport code, landmark) for which to fetch the weather."
      }
    },
    "required": [
      "place"
    ]
  },
  "responses": [
    {
      "type": "string",
      "description": "A string containing the formatted weather information from wttr.in."
    }
  ],
  "errors": [
    {
      "name": "WeatherFetchError",
      "description": "Raised if there is an issue fetching the weather information (e.g., network error, invalid location)."
    }
  ],
  "examples": []
}
```

In [15]:
prompt = toolkit.prompts.function_calling(model)
messages.append({"role": "user", "content": prompt})

In [16]:
task = "What's the weather in Pune and Singapore?"

In [17]:
messages.append({"role": "user", "content": task})

In [18]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

<blockquote>
Okay, the user wants the weather for Pune and Singapore. I have a function `get_weather` that can fetch weather information for a given place. I need to call this function twice, once for each city. Since the two calls are independent, I can call them in parallel.

I will follow all the guidelines outlined in the prompt, specifically:
- Asking for clarification if information is missing (not needed here).
- Using the function specification to understand the function's capabilities.
- Constructing appropriate `function_call` blocks.
- Waiting for the `function_output` before proceeding.
- Providing a final response that fully addresses the user's request.

I will not make any assumptions and will only use the provided function.
</blockquote>



```json
{
	"id": "1",
	"function": "get_weather",
	"parameters": {
		"place": "Pune"
	}
}
```

```json
{
	"id": "2",
	"function": "get_weather",
	"parameters": {
		"place": "Singapore"
	}
}
```

In [19]:
output = await toolkit.code.execute_function("get_weather", {"place": "Pune"})
messages.append(
    {
        "role": "user",
        "content": dedent(f"""\
        ```function_output
        {{
            "id": "pune_weather",
            "result": {output["result"]}
        }}
        ```
    """),
    }
)

In [20]:
output = await toolkit.code.execute_function("get_weather", {"place": "Singapore"})
messages.append(
    {
        "role": "user",
        "content": dedent(f"""\
        ```function_output
        {{
            "id": "singapore_weather",
            "result": {output["result"]}
        }}
        ```
    """),
    }
)

In [21]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

<blockquote>
I have received the weather information for both Pune and Singapore from the `get_weather` function. Now I can construct a final response to the user, providing the weather details for both locations. I will present the information in a clear and concise manner.
</blockquote>



The weather in Pune is patchy rain with a temperature of +24°C (feels like +26°C) and winds at →22km/h.

The weather in Singapore is partly cloudy with a temperature of +27°C (feels like +30°C) and winds at ←9km/h.